In [1]:
# automatically update imports
%load_ext autoreload
%autoreload 2

# For graphs
%matplotlib inline

# snippet for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import loaders

city_ref = 'sant_feilu'

bbox = [41.3700, 2.0203, 41.3973, 2.0774]

pois_df = loaders.load_pois(city_ref, bbox)
categories_kde_dict = loaders.load_grid_kde(city_ref, pois_df, bbox)

Saving data for `['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']` at `/home/martibosch/urbansprawl/urban_analysis/hdfs_store/sant_feilu.h5`
The data has been stored locally with success


Querying locally for `['pois']`
Found ['pois'] stored locally
Querying locally for `['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']`
`['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']` is/are not stored locally. Determining it/them through `get_all_kde_dict` method


In [38]:
import kde
xx, yy = kde._grid_from_bbox(bbox, 0.0015)
arr = categories_kde_dict['kde_global'].values

In [50]:
import numpy as np

def moran_index(xx, yy, f):
    x_row = np.hstack(xx)
    y_row = np.hstack(yy)
    f_row = np.hstack(f)
    f_avg = 1 / np.sqrt((x_row[-1] - x_row[0])**2 + (y_row[-1] - y_row[0])**2)
    phi = [0, 0, 0]
    for i, (x_i, y_i) in enumerate(zip(x_row, y_row)):
        f_i = f_row[i]
        for j, (x_j, y_j) in enumerate(zip(x_row[i+1:-1], y_row[i+1:-1]), start=i+1):
            # print('(%d,%d) against (%d,%d):' % (x_i, y_i, x_j, y_j))
            w = 1.0 / np.sqrt((x_i - x_j)**2 + (y_i - y_j)**2)
            # print('weight for (%d, %d) : %f' % (i, j, w))
            phi[0] += w
            phi[1] += w * ( f_i - f_avg ) * (f_row[j] - f_avg) 
        phi[2] += (f_i - f_avg) ** 2
        # print('completed row ' + str(i) + ' of ' + str(len(x_row)))
    return (float(len(x_row)) / phi[0]) * (phi[1] / phi[2])

def geary_index(xx, yy, f):
    x_row = np.hstack(xx)
    y_row = np.hstack(yy)
    f_row = np.hstack(f)
    f_avg = 1 / np.sqrt((x_row[-1] - x_row[0])**2 + (y_row[-1] - y_row[0])**2)
    phi = [0, 0, 0]
    for i, (x_i, y_i) in enumerate(zip(x_row, y_row)):
        f_i = f_row[i]
        for j, (x_j, y_j) in enumerate(zip(x_row[i+1:-1], y_row[i+1:-1]), start=i+1):
            # print('(%d,%d) against (%d,%d):' % (x_i, y_i, x_j, y_j))
            w = 1.0 / np.sqrt((x_i - x_j)**2 + (y_i - y_j)**2)
            # print('weight for (%d, %d) : %f' % (i, j, w))
            phi[0] += w
            phi[1] += w * ( f_i - f_row[j] ) ** 2
        phi[2] += (f_i - f_avg) ** 2
        # print('completed row ' + str(i) + ' of ' + str(len(x_row)))
    return (.5 * (len(x_row) - 1) / phi[0]) * (phi[1] / phi[2])

In [61]:
xx5, yy5 = np.meshgrid(np.arange(5), np.arange(5))

In [84]:
zz51 = np.array([[0,0,0,0,10],[0,0,0,0,10],[0,0,0,0,0],[0,0,0,0,0],[10,10,0,0,0]])
zz52 = np.array([[0,10,0,0,0],[0,0,0,0,10],[0,0,0,0,0],[0,0,10,0,0],[10,0,0,0,0]])
zz53 = np.array([[0,10,0,0,0],[0,0,0,0,10],[0,0,0,0,0],[0,10,0,0,0],[10,0,0,0,0]])
zz54 = np.array([[0,10,0,0,0],[0,0,0,0,10],[0,0,0,0,0],[0,100,0,0,0],[100,0,0,0,0]])

In [82]:
zz53

array([[ 0, 10,  0,  0,  0],
       [ 0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0],
       [ 0, 10,  0,  0,  0],
       [10,  0,  0,  0,  0]])

In [80]:
geary_index(xx5,yy5,zz52)

0.9038349282493604

In [81]:
geary_index(xx5,yy5,zz53)

0.88473131979518993

In [85]:
geary_index(xx5,yy5,zz54)

0.88752798299049473

In [43]:
def f(x,y): return (x + y)**2
ff = np.vectorize(f)
zz = ff(xx,yy)
zz

array([[ 4,  9, 16],
       [ 9, 16, 25],
       [16, 25, 36]])

In [40]:
moran_index(xx,yy,arr)

0.65517825707838673

15.8550498051


In [19]:
xx, yy = np.meshgrid(np.arange(1,11), np.arange(21,26))

In [23]:
xx

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])

In [22]:
yy

array([[21, 21, 21, 21, 21, 21, 21, 21, 21, 21],
       [22, 22, 22, 22, 22, 22, 22, 22, 22, 22],
       [23, 23, 23, 23, 23, 23, 23, 23, 23, 23],
       [24, 24, 24, 24, 24, 24, 24, 24, 24, 24],
       [25, 25, 25, 25, 25, 25, 25, 25, 25, 25]])

In [28]:
def f(x,y): return x**3 + y
ff = np.vectorize(f)
zz = ff(xx,yy)
zz

array([[  22,   29,   48,   85,  146,  237,  364,  533,  750, 1021],
       [  23,   30,   49,   86,  147,  238,  365,  534,  751, 1022],
       [  24,   31,   50,   87,  148,  239,  366,  535,  752, 1023],
       [  25,   32,   51,   88,  149,  240,  367,  536,  753, 1024],
       [  26,   33,   52,   89,  150,  241,  368,  537,  754, 1025]])

In [17]:
import datetime
ti = datetime.datetime.now()
phi_I = moran_index(xx, yy, arr)
tf = datetime.datetime.now()
print((tf-ti).seconds)

i
(2.0338000000000007, 41.383499999999998)
i
(2.0638000000000019, 41.383499999999998)
i
(2.0353000000000008, 41.384999999999998)
i
(2.0653000000000019, 41.384999999999998)
i
(2.0368000000000008, 41.386499999999998)
i
(2.066800000000002, 41.386499999999998)
i
(2.0383000000000009, 41.387999999999998)
i
(2.068300000000002, 41.387999999999998)
i
(2.0398000000000009, 41.389499999999998)
i
(2.0698000000000021, 41.389499999999998)
i
(2.041300000000001, 41.390999999999998)
i
(2.0713000000000021, 41.390999999999998)
i
(2.0428000000000011, 41.392499999999998)
i
(2.0728000000000022, 41.392499999999998)
i
(2.0443000000000011, 41.393999999999998)
i
(2.0743000000000023, 41.393999999999998)
i
(2.0458000000000012, 41.395499999999998)
i
(2.0758000000000023, 41.395499999999998)
i
(2.0473000000000012, 41.396999999999998)
i
(2.0773000000000024, 41.396999999999998)
1


i
(2.0533000000000015, 41.372999999999998)
i
(2.0248000000000004, 41.374499999999998)
i
(2.0548000000000015, 41.374499999999998)
i
(2.0263000000000004, 41.375999999999998)
i
(2.0563000000000016, 41.375999999999998)
i
(2.0278000000000005, 41.377499999999998)
i
(2.0578000000000016, 41.377499999999998)
i
(2.0293000000000005, 41.378999999999998)
i
(2.0593000000000017, 41.378999999999998)
i
(2.0308000000000006, 41.380499999999998)
i
(2.0608000000000017, 41.380499999999998)
i
(2.0323000000000007, 41.381999999999998)
i
(2.0623000000000018, 41.381999999999998)


15.8550498051
i
(2.0203000000000002, 41.369999999999997)
i
(2.0503000000000013, 41.369999999999997)
i
(2.0218000000000003, 41.371499999999997)
i
(2.0518000000000014, 41.371499999999997)
i
(2.0233000000000003, 41.372999999999998)
